# LDPC Codes

Encoding method - reduce parity check matrix to row-echelon form, then matrix multiplication encoding

Decoding method - loopy belief propagation algorithm for BSC

In [ ]:
import random
import numpy as np
import numba as nb
import pandas as pd
from tqdm import tqdm

### Encoder

In [184]:
H = np.array([[1, 1, 1, 1, 0, 0],
              [0, 0, 1, 1, 0, 1],
              [1, 0, 0, 1, 1, 0]])

In [185]:
def gaussian_elimination(H):
    """
    Gaussian elimination of a parity check matrix H in F2.

    H: array_like
        Parity check matrix.

    H_redcued: array_like
        Reduced parity check matrix.
    """
    H_reduced = H.copy()
    n, m = H.shape
    
    for i in range(n):

        # find the row with a leading 1 in the i-th column
        for j in range(i, n):
            if H_reduced[j, i] == 1:
                break 

        # swap the rows to make the leading 1 appear on the diagonal
        H_reduced[[i, j], :] = H_reduced[[j, i], :]

        # eliminate the leading 1 from other rows in the same column
        for j in range(i+1, n):
            if H_reduced[j, i] == 1:
                H_reduced[j, :] = (H_reduced[j, :] ^ H_reduced[i, :])

    # reorder the columns to make the identity matrix appear on the left
    indices = np.where(H_reduced.sum(axis=0) == 1)[0]
    for i, index in enumerate(indices):
        H_reduced[:, [i, index]] = H_reduced[:, [index, i]]

    return H_reduced

In [186]:
H_reduced = gaussian_elimination(H)
H, H_reduced

(array([[1, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 0, 1],
        [1, 0, 0, 1, 1, 0]]),
 array([[1, 0, 0, 1, 1, 1],
        [0, 1, 0, 0, 1, 1],
        [0, 0, 1, 1, 0, 1]]))

In [187]:
def generator_matrix(H):
    """
    Construct a generator matrix G from a parity check matrix H.

    H: array_like
        Parity check matrix.

    G: array_like
        Generator matrix.
    """
    H_reduced = gaussian_elimination(H)
    n, k = H_reduced.shape

    G = np.zeros((k, n), dtype=int)
    G[:n, :] = np.eye(n, dtype=int)
    G[n:, :] = H_reduced[:, n:].T

    return G, H_reduced

In [188]:
generator_matrix(H_reduced)

(array([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 1],
        [1, 1, 0],
        [1, 1, 1]]),
 array([[1, 0, 0, 1, 1, 1],
        [0, 1, 0, 0, 1, 1],
        [0, 0, 1, 1, 0, 1]]))

In [189]:
def binary_symmetric_channel(x, p):
    """
    Define passage through a binary symmetric channel with noise p.

    x : int
        Input bit.
    p : float
        Channel error probability.

    y : int
        Output bit after applying the channel.
    """
    return x if np.random.rand() > p else 1 - x

In [190]:
def encoder(G, t, p):
    """
    Encode a message t using a generator matrix G and a channel p in F2.

    G : array_like
        Generator matrix.
    t : array_like
        Message to encode.
    p : float
        Channel flip probability.

    x : array_like
        Encoded message.
    """
    
    x = np.dot(G.T, t) & 1  # Bitwise AND operation in F2
    return np.array([binary_symmetric_channel(xi, p) for xi in x])

In [191]:
G = generator_matrix(H_reduced)[0]
t = np.array([1, 0, 1, 0, 0, 0]).reshape(-1, 1)
x = encoder(G, t, 0.1)
x

array([[1],
       [0],
       [0]])

### Loopy Belief Propagation Decoder

In [197]:
H_data = pd.read_csv('h1.csv', header=None).values
H_data[:5]

array([['0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [198]:
# transform H_data to parity check matrix
H = np.array([np.array([int(i) for i in H_data[j][0].split()]) for j in range(len(H_data))])

In [199]:
y_data = pd.read_csv('y1.csv', header=None).values
y_data[:5]

array([[0],
       [1],
       [0],
       [0],
       [0]])

In [200]:
# transform y_data to row vector
y = y_data.reshape(1, -1)[0]

In [201]:
@nb.njit(fastmath=True, parallel=True)
def init_p(y, p):
    """
    Initialize probability of each bit in received codeword, y, being 1.

    y : array_like
        Received codeword.
    p : float
        Channel flip probability.

    prob : array_like
        Probability of each bit in y being 1.
    """
    
    # initialize probability of each bit in received codeword, y, being 1
    n = len(y)
    prob = np.zeros((n))

    # iterate over each bit in y
    for i in nb.prange(n):
        # if bit is 0, probability of bit being 1 is probability it flipped
        if y[i] == 0:
            prob[i] = p
        # if bit is 1, probability of bit being 1 is probability it did not flip
        else:
            prob[i] = 1-p # probability of bit being 1 is probability it did not flip

    return prob

In [202]:
@nb.njit(fastmath=True, parallel=True)
def init_q(H, n, m, y, p):
    """
    Initialize messages q_{ij} for each check node j as n x m x 2 array.

    H : array_like
        Parity check matrix.
    n : int
        Number of bits.
    m : int
        Number of check nodes.
    y : array_like
        Received codeword.
    p : float
        Channel flip probability.
        
    q : array_like
        Messages q_{ij} for each check node j as n x m x 2 array.
    """
    
    # initialize messages q_{ij} for each check node j as n x m x 2 array
    # [:,:,0] is probability of bit i being 0 
    # [:,:,1] is probability of bit i being 1
    q = np.zeros((n, m, 2))

    # update messages q_{ij} based on received codeword, y, and probability of each bit being 1, p
    for j in nb.prange(m):
        for i in nb.prange(n):
            if H[i,j] == 1: # if bit i is connected to check node j
                if y[i] == 0: 
                    # if bit i is 0:
                    # then probability of bit being 0 is probability it did not flip
                    # and probability of bit being 1 is probability it flipped
                    q[i, j, 0] = 1 - p[i]
                    q[i, j, 1] = p[i]
                else:
                    # if bit i is 1:
                    # then probability of bit being 0 is probability it flipped
                    # and probability of bit being 1 is probability it did not flip
                    q[i, j, 0] = p[i]
                    q[i, j, 1] = 1 - p[i]
                    
    return q

In [203]:
p = init_p(y, 0.1)
q = init_q(H.T, H.shape[1], H.shape[0], y, p)

In [204]:
H.shape, y.shape

((750, 1000), (1000,))

In [205]:
nb.njit(fastmath=True, parallel=True)
def decoder(received_codeword, H, noise, max_iter=20):
    """
    Decode a received codeword using a parity check matrix H and a channel noise.

    received_codeword : array_like
        Received codeword.
    H : array_like
        Parity check matrix.
    noise : float
        Channel noise.
    max_iter : int, optional
        Maximum number of iterations.

    x_hat : array_like 
        Decoded codeword.
    """
    
    n, m = H.shape  # n: number of check nodes, m: number of variable nodes
    x_hat_prev = np.zeros(len(received_codeword))  # initialise an empty decoded codeword
    p = init_p(received_codeword, noise)  # initialise the messages p_{i} - probability of bit i in the receievd codeword being 1
    q = init_q(H, n, m, received_codeword, p)  # initialise the messages q_{ij} - var i to check j
    r = np.zeros((m, n, 2))  # initialise the messages r_{ji} - check j to var i

    for iter in tqdm(nb.prange(1, max_iter+1)):

        # update messages r_{ji}
        for i in nb.prange(n):  # iterate over all variable nodes
            for j in nb.prange(m): # iterate over all check nodes
                related_vars = np.where(H[:, j] == 1)[0] # find the variable nodes related to the current check node
                related_vars = related_vars[related_vars != i]  # exclude the current variable node
                r[j,i,0] = 0.5 + 0.5 * np.prod(1 - 2*q[related_vars, j, 1])
                r[j,i,1] = 1 - r[j,i,0]

        # update messages q_{ij}
        for j in nb.prange(m): # iterate over all check nodes
            for i in nb.prange(n): # iterate over all variable nodes
                related_checks = np.where(H[i, :] == 1)[0] # find the check nodes related to the current variable node
                related_checks = related_checks[related_checks != j]  # exclude the current check node
                q[i,j,0] = (1-p[i]) * np.prod(r[related_checks, i, 0])
                q[i,j,1] = p[i] * np.prod(r[related_checks, i, 1])
                # normalise the messages since they represent probabilities
                qk = 1 / (q[i, j, 0] + q[i, j, 1]) 
                q[i,j,0] *= qk
                q[i,j,1] *= qk

        # calculate the bitwise estimate of every bit
        x_hat = np.zeros(len(received_codeword))  # initialise an empty decoded codeword
        for i in nb.prange(m): # iterate over all check nodes
            related_checks = np.where(H[i, :] == 1)[0] # find the check nodes related to the current variable node
            # product over all messages in likelihood calculations - no exclusions
            Q_0 = (1-p[i]) * np.prod(r[related_checks, i, 0])
            Q_1 = p[i] * np.prod(r[related_checks, i, 1])
            # normalize the messages since they represent probabilities
            Qk = 1 / (Q_0 + Q_1)
            Q_0 *= Qk
            Q_1 *= Qk
            x_hat[i] = 1 if Q_1 > Q_0 else 0 # maximum likelihood decoding


        x_hat = x_hat.astype(int)
        H = H.astype(int)

        # early stopping conditions 
        if (np.bitwise_xor.reduce([np.dot(row, x_hat.T) % 2 for row in H.T]) == 0 # Hx = 0 in F2 (valid codeword)
            and not np.array_equal(x_hat, np.zeros(len(x_hat))) # x != 0 (non-zero codeword)
            and np.array_equal(x_hat, x_hat_prev)): # x == x_prev (convergence)
            return x_hat, (0, iter)
        
        x_hat_prev = x_hat.copy() # update the previous decoded codeword

    return x_hat.astype(int), -1

In [206]:
# @nb.njit()
# def init_log_q(H, n, m, y, p):
#     # Initialize the messages for each check node j
#     log_q = np.zeros((n, m))
#     for i in nb.prange(n):
#         for j in nb.prange(m):
#             if H[i, j] == 1:
#                 log_q[i, j] = (-1)**y[i]*np.log((1-p)/p)
#     return log_q

In [207]:
# @nb.njit()
# def init_log_c(y, p):
#     # BSC channel log likelihood ratio
#     log_c = np.zeros(len(y))
#     for i in nb.prange(len(y)):
#         log_c[i] = (-1)**y[i]*np.log((1-p)/p)
#     return log_c

In [208]:
# @nb.njit()
# def log_decoder(received_codeword, H, p, max_iter=20):
#     n, m = H.shape
#     x_hat = np.zeros(len(received_codeword))
#     L_c = init_log_c(received_codeword, p)
#     L_q = init_log_q(H, n, m, received_codeword, p) 
#     L_r = np.zeros((m, n))

#     for _ in range(max_iter):

#         # update L_r
#         for j in nb.prange(m):
#             for i in nb.prange(n):
#                 related_vars = np.where(H[:, j] == 1)[0] # Find the related variable nodes
#                 related_vars = related_vars[related_vars != i]  # Exclude the current variable node
#                 alpha = np.sign(L_q[related_vars, j])
#                 beta = np.abs(L_q[related_vars, j])
#                 # L_r[j, i] = np.prod(alpha)*np.min(beta, axis=0)
#                 sum_comp = -np.log(np.tanh(np.sum(-np.log(np.tanh(beta)/2), axis=0)/2))
#                 L_r[j, i] = np.prod(alpha)*sum_comp

#         # update L_q
#         for i in nb.prange(n):
#             for j in nb.prange(m):
#                 related_checks = np.where(H[i, :] == 1)[0] # Find the related check nodes
#                 related_checks = related_checks[related_checks != j] # Exclude the current check node
#                 L_q[i, j] = L_c[i] + np.sum(L_r[related_checks, i])

#         # calculate the bitwise estimate of every bit
#         for i in nb.prange(n):
#             related_checks = np.where(H[i, :] == 1)[0]
#             L_Q = L_c[i] + np.sum(L_r[related_checks, i])
#             x_hat[i] = 1 if L_Q < 0 else 0

#         # check if the estimated codeword is valid
#         # if np.array_equal(np.dot(x_hat, H) % 2, np.zeros(m)):
#         #     return x_hat, 0, i
        
#     return x_hat, -1

In [209]:
decoded_signal, result = decoder(y.T, H.T, 0.1, 20)

 50%|█████     | 10/20 [01:40<01:40, 10.03s/it]


In [212]:
decoded_signal, result

(array([0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
        0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
        1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
        1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1,
        1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
        0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
        1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1,
        0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 

In [213]:
str_bits = ''.join([str(int(i)) for i in decoded_signal[:8]])

In [214]:
str_bits

'01001000'

In [215]:
def recover_english_message(decoded_signal):

    extracted_bits = decoded_signal[:248] # extract the first 248 bits
    str_bits = ''.join([str(int(i)) for i in extracted_bits]) # convert the bits to a string
    
    # convert the 248 bits into a sequence of 31 ASCII symbols
    ascii_symbols = []
    for i in range(0, 248, 8):  # step by 8 bits to get each ASCII symbol
        ascii_value = chr(int(str_bits[i:i+8], 2)) # convert the 8 bits to an ASCII symbol
        ascii_symbols.append(ascii_value) # append the ASCII symbol to the list of ASCII symbols

    original_message = ''.join(ascii_symbols) # join the list of ASCII symbols into a string to get the original message
    
    return original_message

In [216]:
# original encoded message in ASCII
encoded_message = recover_english_message(y)
encoded_message

'Dapxy0ÌnlhdAisa(Dmy4r9&@iVhd@¢-'

In [217]:
# decoded message in ASCII - converged in 11 iterations
decoded_message = recover_english_message(decoded_signal.astype(int))
decoded_message

'Happy Holidays! Dmitry&David :)'